In [1]:
import os
import shutil
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import uuid
import cv2
from PIL import Image
import tensorflow as tf
# import tensorflow_io as tfio

# from pydicom import dcmread 

from duke_dbt_data import dcmread_image,draw_box

In [4]:
def merge_datasets(source: str):
    df_paths = pd.read_csv("file-paths copy.csv")
    df_labels = pd.read_csv("labels-train.csv")
    df_boxes = pd.read_csv("BCS-DBT boxes-train-v2.csv")
    df_true = df_paths.merge(df_labels, how='inner')
    df_true = df_true.merge(df_boxes, how='inner')
    df_true['descriptive_path'] = source + df_true['descriptive_path'].astype(str)
    df_paths['descriptive_path'] = source + df_paths['descriptive_path'].astype(str)

    #WINDOWS
    #df_true['descriptive_path'] = df_true['descriptive_path'].str.replace('/',R'\\')
    #MAC
    df_true['descriptive_path'] = df_true['descriptive_path']
    df_true[(df_true[['Cancer','Benign','Actionable']] == 1).any(axis=1)]
    ##REMOVENDO TRUE POSITIVE DO DATASET PATH PARA FILTRAR IMAGENS SEM LABEL
    df_regular = df_paths[~df_paths['PatientID'].isin(df_true['PatientID'])]
    
    return df_true,df_regular 



In [ ]:
import concurrent.futures

rootdir = '/Volumes/My Passport/MRI TP images/manifest-1677805544485/'
destination = '/Users/lucas.wojahn/Documents/GitHub/duke-dbt-data/data/train2/'

##NÃO ESQUECE DE PASSAR O SLICE_INDEX (AUMENTA A QUALIDADE)

def process_image(path, 
    view, 
    slice_index, 
    destination, 
    file_name,
    row
):
    image = dcmread_image(fp=path, view=view, index=slice_index)
    file_location = destination+'/'+file_name
    plt.imsave(file_location, image,  format="jpg", cmap=plt.cm.gray)
    return file_location,row

def write_annotation_file(
    df: pd.DataFrame,
    annotation_file: str
    ):

    # LEMBRA DE TROCAR A PORRA DO [0] FIXO NO DATAFRAME
    img_width, img_height = df['Width'], df['Height']
    X, Y = df['X'], df['Y']

    image = cv2.imread(annotation_file)

    total_image_height = image.shape[0]
    total_image_width =  image.shape[1]

    normalized_x = X/total_image_width
    normalized_y = Y/total_image_height
    normalized_width = img_width/total_image_width
    normalized_height = img_height/total_image_height
  
    annotation_file = destination + annotation_file.replace(".jpg", ".txt")
    print(annotation_file)

    f = open(annotation_file, "w")

    if df['Benign'] == 1:
        classe = 0
    elif df['Cancer'] == 1:
        classe = 1
    elif df['Actionable'] == 1:
        classe = 2

    if classe is not None:
        # print(f"salvando {classe} {X} {Y} {img_width} {img_height}")
        f.write(f"{classe} {normalized_x} {normalized_y} {normalized_width} {normalized_height}")

    f.close()


def process_TP(source: str, destino: str):
    
    df_true, df_regular = merge_datasets(source)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for index, row in df_true.iterrows():
            #NOMEANDO ARQUIVOS
            name = str(row['PatientID'])
            slice_index = row['Slice']
            uuid_name = str(uuid.uuid4())

            annotation_file = name+'-'+uuid_name+'.jpg'
            # write_annotation_file(df=row, annotation_file=annotation_file)

            # MAC
            split = row['descriptive_path'].split('/')
            split[-2] = split[-2].replace('-','-NA-')
            path = '/'.join(split)
            view = row["View"]

            futures.append(executor.submit(process_image, path, view, slice_index, destino, annotation_file, row))
        for future in concurrent.futures.as_completed(futures):
            try:
                # file_location = future.result()
                file_location,row  = future.result()
                write_annotation_file(df=row, annotation_file=file_location)
            except Exception as e:
                print(f"Exception: {e}")

    df_true.drop(index, inplace=True)

process_TP(source=rootdir,destino=destination)


In [5]:
## VERSAO MELHORADA
import concurrent.futures
import cv2
import imutils
import pybboxes as pbx

rootdir = '/Volumes/My Passport/MRI TP images/manifest-1677805544485/'
destination = '/Users/lucas.wojahn/Documents/GitHub/duke-dbt-data/data/train2/'
resized = '/Users/lucas.wojahn/Documents/GitHub/duke-dbt-data/data/resized/'

def verifica_classe(df):
    classe = None

    if df['Benign'] == 1:
        classe = 0
    elif df['Cancer'] == 1:
        classe = 1
    elif df['Actionable'] == 1:
        classe = 2
    
    return classe

def resize_image(image_path, nome_arquivo):
    img = cv2.imread(image_path)
    height, width, channels = img.shape
    max_size = 416

    # Determine the scaling factor to resize the image while preserving aspect ratio
    # scaling_factor_Y,scaling_factor_X = 0,0
    # if height > width:
    #     scaling_factor_Y = max_size / height
    # else:
    #     scaling_factor_X = max_size / width

    output = resized + nome_arquivo
    # Resize the image using the calculated scaling factor
    resized_img = imutils.resize(img, width=416)
    # resized_img = cv2.resize(img, None, fx=scaling_factor_X, fy=scaling_factor_Y, interpolation=cv2.INTER_AREA)
    cv2.imwrite(output, resized_img)

def normalize_image(image_path, nome_arquivo ,row):
    n_annotation_file = nome_arquivo.replace('.jpg','.txt')
    X, Y = row['X'], row['Y']
    img_width, img_height = row['Width'], row['Height']

    image = cv2.imread(image_path)

    total_image_height = image.shape[0]
    total_image_width =  image.shape[1]

    voc_bbox = (X, Y, img_width, img_height)
    W, H = total_image_width, total_image_height  # WxH of the image
    normalized_x,normalized_y,normalized_width,normalized_height = pbx.convert_bbox(voc_bbox, from_type="coco", to_type="yolo", image_size=(W,H))


    classe = verifica_classe(row)

    with open(os.path.join(resized, n_annotation_file), "w") as f:
        f.write(f'{classe} {normalized_x} {normalized_y} {normalized_width} {normalized_height}')

def process_image(path, view, slice_index, destination, file_name, row):
    image = dcmread_image(fp=path, view=view, index=slice_index)
    file_location = destination + '/' + file_name
    plt.imsave(file_location, image, format="jpg", cmap=plt.cm.gray)
    return file_location

def write_annotation_file(df: pd.DataFrame, annotation_file: str):
    img_width, img_height = df['Width'], df['Height']
    X, Y = df['X'], df['Y']

    annotation_file = destination + annotation_file.replace(".jpg", ".txt")

    f = open(annotation_file, "w")

    classe = verifica_classe(df)

    if classe is not None:
        f.write(f"{classe} {X} {Y} {img_width} {img_height}")

    f.close()

def process_row(row):
    name = str(row['PatientID'])
    slice_index = row['Slice']
    uuid_name = str(uuid.uuid4())

    annotation_file = name+'-'+uuid_name+'.jpg'
    write_annotation_file(df=row, annotation_file=annotation_file)

    # MAC
    split = row['descriptive_path'].split('/')
    split[-2] = split[-2].replace('-','-NA-')
    path = '/'.join(split)
    view = row["View"]

    file_location = process_image(path, view, slice_index, destination, annotation_file, row)
    return file_location,annotation_file, row

def process_TP(source: str, destino: str):
    df_true, df_regular = merge_datasets(source)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for _, row in df_true.iterrows():
            futures.append(executor.submit(process_row, row))

        for future in concurrent.futures.as_completed(futures):
            try:
                file_location, nome_arquivo, row = future.result()
                normalize_image(file_location, nome_arquivo, row)
                resize_image(file_location, nome_arquivo)

            except Exception as e:
                print(f"Exception: {e}")

process_TP(source=rootdir, destino=destination)


In [ ]:
##PROCESSA IMAGENS QUE NÃO POSSUEM TRUE POSITIVE
def processa_normal(source: str, destino: str):

    df_true, df_paths = merge_datasets(source)

    for index, row in df_paths.iterrows():
        name = str(row['PatientID'])
        uuid_name = str(uuid.uuid4())
        annotation_file = name+'-'+uuid_name+'.jpg'
        split = row['descriptive_path'].split('/')
        split[-2] = split[-2].replace('-','-NA-')
        path = '/'.join(split)
        view = row["View"]
        image = dcmread_image(fp=path, view=view)
        plt.imsave(destination+'/'+annotation_file, image[0],  format="jpg", cmap=plt.cm.gray)
        df_paths.drop(index, inplace=True)
        
processa_normal(source=rootdir,destino=destination)


In [ ]:
####ADICIONANDO MULTITHREAD A FUNÇÃO
import concurrent.futures

df_true, df_paths = merge_datasets(rootdir)

def process_image(path, view, destination, annotation_file):
    image = dcmread_image(fp=path, view=view)
    plt.imsave(destination+'/'+annotation_file, image[0],  format="jpg", cmap=plt.cm.gray)
    return annotation_file

def processa_normal(source: str, destino: str):

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for index, row in df_paths.iterrows():
            name = str(row['PatientID'])
            uuid_name = str(uuid.uuid4())
            annotation_file = name+'-'+uuid_name+'.jpg'
            split = row['descriptive_path'].split('/')
            split[-2] = split[-2].replace('-','-NA-')
            path = '/'.join(split)
            view = row["View"]
            futures.append(executor.submit(process_image, path, view, destino, annotation_file))
        for future in concurrent.futures.as_completed(futures):
            try:
                result = future.result()
            except Exception as e:
                print(f"Exception: {e}")
    df_paths.drop(index, inplace=True)

processa_normal(source=rootdir,destino=destination)

In [ ]:
from PIL import Image, ImageDraw

# Load the image
image = Image.open('example.jpg')

# Create a rectangle object with the desired coordinates
x, y, width, height = 100, 100, 200, 300
bbox = (x, y, x+width, y+height)

# Draw the bounding box on the image
draw = ImageDraw.Draw(image)
draw.rectangle(bbox, outline='red')

# Save the modified image
image.save('example_bbox.jpg')

In [ ]:
process_TP(source=rootdir,destino=destination)
processa_normal(source=rootdir,destino=destination)